# Introducción a bases de datos NoSQL 2

## Segunda parte: Conceptos

## Replicación

Ver más: https://docs.mongodb.com/manual/replication

Un *replica set* —conjunto de réplicas— en MongoDB es un grupo de procesos que mantienen el mismo *dataset*. Cada proceso corre en una máquina distinta (nodo) dentro de un grupo de máquinas (*cluster*).

Las réplicas proveen **redundancia y tolerancia a fallas**, son la base de todos los *deploys* en producción.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/replica_compass.png)

### Nodo primario

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/replica_set.svg)

El **nodo primario** recibe todas las operaciones de escritura y lectura; solo puede haber uno.

### Nodos secundarios

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/replica_secondary.svg)

Los **nodos secundarios** replican las operaciones del primario en sus datasets, de tal manera de reflejarlo.

Si el nodo primario no está disponible, un secundario iniciará ocupar su lugar.

### Tolerancia a fallas

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/replica_failover.svg)

Cuando un primario no se comunica con los otros nodos dentro de un cierto periodo (10 segundos por defecto), un secundario llama a elección para nominarse como el nuevo primario.

El *replica set* **no puede procesar operaciones de escritura** durante la elección.

### Write concern

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/replica_concern.svg)

Las operaciones de escritura requieren un **reconocimiento de persistencia**. Este reconocimiento es configurable; cuando su valor es "mayoría", la escritura se confirma cuando las operaciones se hayan propagado en la mayoría de los nodos.

### Preferencia de lectura

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/eventual_consistency.svg)

Opcionalmente, es posible habilitar la lectura desde nodos secundarios para aumentar la capacidad de lectura.

MongoDB se vuelve **eventualmente consistente**, permitiendo leer datos potencialmente desactualizados.

### Distribución geográfica

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/replica_zones.svg)

Distribuir nodos en distintos centros de datos mejora la redundancia y la tolerancia a fallas.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/replica_read_preference.svg)

Los clientes pueden configurarse para preferir leer desde nodos secundarios para mejorar la latencia.

## Fragmentación

Ver más: https://docs.mongodb.com/manual/sharding

*Sharding* —fragmentación— en MongoDB es un método para distribuir los datos a lo largo de múltiples máquinas.

Es útil cuando el *dataset* es muy grande y/o la cantidad de operaciones es muy alta. Un conjunto de datos de trabajo que supera la RAM del sistema estresa la lecto-escritura de los discos. Una alta tasa de consultas exhausta al CPU.

Dividir la carga entre múltiples servidores y añadir servidores según necesidad para aumentar la capacidad es lo que se conoce como **escalibilidad horizontal**.

Incrementar las capacidades de procesamiento, memoria y almacenamiento de un único servidor tiene el nombre de **escalibilidad vertical**; en la práctica esta escalabilidad es limitada.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/shard_collections.svg)

Las colecciones fragmentadas son particionadas y distribuidas en los *shards* —nodos— del *cluster*.

Las colecciones no particionadas son guardadas en un *shard* primario.

### Conexión

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/shard_connection.svg)

Los clientes jamás deberían conectarse a un fragmento (nodo que corre un proceso `mongod`) directamente. Deben hacerlo a través de un nodo enrutador (corre un proceso `mongos`).

### Cluster fragmentado

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/shard_cluster.svg)

Consiste en los siguientes componentes:
* Fragmentos. Cada uno de estos nodos puede deployarse como un *replica set*.
* Enrutadores.
* Servidores de configuración.

### Shard keys

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/shard_chunks.svg)

La *shard key* es un **índice simple o compuesto** que determina la distribución de los documentos de una colección entre los fragmentos del cluster.

MongoDB divide los valores de la llave en rangos yuxtapuestos. Cada rango es asociado a un pedazo (*chunk*). MongoDB intenta distribuirlos de manera equitativa entre los *shards*.

La llave debería tener los siguientes atributos:

* Alta cardinalidad.
* Baja frecuencia de valores.
* No ser monótona.

Para cuando no es posible cumplirlos, existe otra estrategia llamada [hashed sharding](https://docs.mongodb.com/manual/core/hashed-sharding/).

### Chunks

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/shard_chunk_split.svg)

Los *chunks* consisten es subconjuntos de datos. Poseen un tamaño máximo configurable (64 MB por defecto).

MongoDB despedaza el *chunk* en caso de crezca más allá del tamaño máximo. En el caso opuesto puede unirlos.

Ambas operaciones llevan a una **distribución despareja** de la colección a los largo de los fragmentos.

### Balanceador

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/shard_chunk_migration.svg)

El balanceador es un proceso de fondo que administra las migraciones de *chunks*.

Si la diferencia en el número de pedazos entre el *shard* más ocupado y el *shard* más vacío supera determinado umbral, el balanceador realiza migraciones para distribuir los datos de manera equitativa.

### Operaciones dirigidas

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/shard_targeted.svg)

En un entorno fragmentado, las consultas más rápidas son aquellas que el enrutador puede dirigir a un único *shard* utilizando la *shard key*.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/shard_broadcast.svg)

Para aquellas consultas que no incluyen la *shard key*, el enrutador debe consultar a todos los fragmentos.

## Teorema CAP

Dice que es **imposible para una base de datos distribuida** cumplir simultáneamente con más dos de las siguientes tres garantías:

* Consistencia
* Disponibilidad
* Tolerancia a la partición

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/cap_parts.png)

Fuente: https://cryptographics.info/cryptographics/blockchain/cap-theorem

Cuando sucede una **partición de red** (desconexión de nodos) es necesario decidir:

* Cancelar la operación, por ende perder disponibilidad pero asegurar la consistencia.
* Proceder con la operación, mantener la disponibilidad a riesgo de inconsistencias.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/cap_triangle.png)

### MongoDB

* Por defecto, todas las operaciones de lectura y escritura van al nodo primario — **consistencia fuerte**.
* Maneja fallas en la red, guardando la misma data en nodos secundarios — **tolerancia a la partición**.
* Compromete la **disponibilidad** durante la votación de un nuevo nodo primario.

El *Domain Name System* (DNS) de internet es un buen ejemplo de sistema de alta disponibilidad y consistencia eventual.

## Características de las bases NoSQL

Naturaleza distribuida
* Escalabilidad horizontal (almacenamiento y cómputo)
* Tolerancia a la partición
* Disponibilidad (solo algunas DBs)

Modelos de datos flexibles
* Consultas más rápidas (datos que se acceden juntos se guardan juntos)
* Facilidad de desarrollo

### Posibles inconvenientes

* Falta de transacciones entre múltiples registros.
* Duplicación de información.

## Tipos de bases NoSQL

Hay cuatro grandes tipos:
* documentos,
* llave-valor,
* columnares,
* grafos.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/nosql_types.png)

*Fuente*: https://itsmecevi.github.io/dwbi

### Documentos

Almacenan colecciones de documentos similares a objetos JSON (*JavaScript object notation*). Estos objetos suelen alinearse con los objetos que existen en el código de la aplicación.

En estructura, los documentos son similares pero no necesarimente iguales. Es posible trabajar con estructuras complejas (documentos anidados).

Estas bases permiten recuperar documentos y generar índices en base a su contenido; pueden procesar consultas complejas.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/mongodb.png)

*Fuente*: https://www.mongodb.com/document-databases

#### Casos de uso

Las bases de datos de documentos están pensadas para datos con estructura compleja y variable.

* Uso general
* Aplicaciones web

#### Bases populares

* MongoDB
* CouchDB
* DynamoDB (AWS)
* Firestore (GCP)

### Llave-valor

Almacenan colecciones de pares llave-valor, como un diccionario.

La llave funciona como identificador único, los valores pueden ser de **cualquier tipo**: estas bases tratan a los datos que contienen como BLOBs (*binary large object*) — 1s y 0s sin estructura. Es responsabilidad de la aplicación entender cómo interpretarlos.

Debido a que los BLOBs son opacos, estas bases
* no realizan búsquedas en base al contenido de los valores,
* ni establecen otro índice más que para las llaves.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/redis.png)

*Fuente*: https://redislabs.com/blog/get-sql-like-experience-redis

#### Casos de uso

Las bases de datos llave-valor son la opción más eficiente cuando las consultas para recuperar los datos no son complejas.

* Almacenamiento caché
* Colas de mensajes
* Manejo de sesiones

#### Bases populares

* Redis
* Riak
* DynamoDB (AWS)
* Firestore (GCP)

### Columnares

En apariencia muy similares a las bases de datos relacionales; en vez de agrupar las columnas en tablas, **cada columna es almacenada por separado**, lo que permite leer solo las columnas que se necesitan en vez de toda la tabla.

Las columnas guardan datos del **mismo tipo**. Las filas pueden tener **distintas columnas**, las tablas son esparzas.

Son consideradas bases llave-valor de dos dimensiones. Los ids de las filas y los nombres de las columnas son los únicos índices; los valores son opacos.

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/columnar.png)

*Fuente*: https://bi-insider.com/portfolio-item/techniques-of-data-warehouse-performance-optimization

#### Casos de uso

Las bases de datos columnares son óptimas para lecturas rápidas de grandes volúmenes de datos y agregaciones de columnas.

* *Big Data*
* Series temporales. Ejemplos: IoT, telemetría, valores mercados. 
* *Logs*. Ejemplos: monitoreo de systemas, datos de usuarios.

#### Bases populares

* Cassandra
* HBase
* Redshift (AWS)
* Bigtable (GCP)

### Grafos

Tienen un parecido a las bases de datos de documentos, con hincapié en las relaciones entre documentos.

Los documentos suelen ser entidades (personas, lugares, cosas); las relaciones son semánticamente relevantes (pertenece a, compró un) y pueden tener atributos (distancia, tiempo, costo).

Ciertas operaciones son mucho más sencillas utilizando estas bases por la manera en la que enlazan y agrupan los datos. 

![](https://raw.githubusercontent.com/matiasbattocchia/clases-aprendizaje-automatico/master/nosql/img/graph.png)

*Fuente*: https://towardsdatascience.com/graph-databases-whats-the-big-deal-ec310b1bc0ed

#### Casos de uso

Las bases de datos de grafos sobresalen a la hora de "atravesar" una red de documentos en busca de patrones.

* Redes sociales
* Detección de fraude
* Sistemas de recomendación
* Bases de conocimiento, como [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)

#### Bases populares

* Neo4j
* ArangoDB